In [24]:
import re, random, torch, pickle, gc, os, sklearn
import optuna
import lightgbm as lgb
import polars as pl
import pandas as pd
import numpy as np
from pathlib import Path
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
## Sklearn package
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor 
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import VotingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error 
from scipy.stats import skew, kurtosis
import ctypes
import math
import warnings
warnings.filterwarnings("ignore")
pd.options.display.max_rows = 999
pd.options.display.max_colwidth = 99

In [2]:
INPUT_DIR = './'
logs = pl.scan_csv(f'{INPUT_DIR}/train_logs.csv')

In [3]:
logs.head()

<LazyFrame [11 cols, {"id": Utf8 … "word_count": Int64}] at 0x1EDF77B6EB0>

In [22]:
def tfidf_by_values(colname, used_cols):
    fts = logs.select(pl.col('id').unique(maintain_order=True))
    total_rows =fts.collect().shape[0]
    
    for i, col in enumerate(used_cols):
        # 计算每个值出现的次数
        tmp_logs =logs.group_by('id').agg(
            pl.col(colname).is_in([col]).sum().alias(f'{colname}_{i}_cnt')
        )
        fts = fts.join(tmp_logs, on='id', how='left')
        
        # 计算 IDF
        
        
        idf = total_rows / (logs.filter(pl.col(colname) == col).collect().shape[0] + 1)
        idf=math.log(idf)
        idf = pl.lit(idf)

        # 计算 TF-IDF 并更新 LazyFrame
        cnts = fts[f'{colname}_{i}_cnt']
        tfidf_col = ((1 + pl.lit(logs[colname]).ln() / cnts) * idf).alias(f'{colname}_{i}_tfidf')
        fts = fts.join(tfidf_col, on='id', how='left')

    return fts

In [23]:
activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
     
#=== Create the feature columns using count by values ===
df =tfidf_by_values('activity', activities) # Create 'activity' column


AttributeError: 'Expr' object has no attribute 'ln'

In [ ]:
# 每一个product过程中每分钟按下的event
    def create_keys_pressed_feats(self):
        logs = self.logs.copy()
        temp_df = logs[logs['activity'].isin(['Input', 'Remove/Cut'])].groupby(['id']).agg(keys_pressed=('event_id', 'count')).reset_index()
        temp_df_2 = logs.groupby(['id']).agg(min_down_time=('down_time', 'min'), max_up_time=('up_time', 'max')).reset_index()
        temp_df = temp_df.merge(temp_df_2, on='id', how='left')
        temp_df['keys_per_second'] = temp_df['keys_pressed'] / ((temp_df['max_up_time'] - temp_df['min_down_time']) / 1000)
        return temp_df[['id', 'keys_per_second']]